In [5]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


Kaggle environment setup & data listing

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

niblabel librar


libraries


In [7]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00:00:0100:01
  Attempting unin

 Create data dictionary for BraTS 2020

In [8]:
from glob import glob
import os

base_path = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
subjects = sorted(os.listdir(base_path))

data_dicts = []
for sub in subjects:
    sub_path = os.path.join(base_path, sub)
    if os.path.isdir(sub_path):
        data_dicts.append({
            "image": [
                os.path.join(sub_path, f"{sub}_t1.nii"),
                os.path.join(sub_path, f"{sub}_t1ce.nii"),
                os.path.join(sub_path, f"{sub}_t2.nii"),
                os.path.join(sub_path, f"{sub}_flair.nii"),
            ],
            "label": os.path.join(sub_path, f"{sub}_seg.nii")
        })


Define preprocessing & augmentation pipeline

In [ ]:
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Compose, ScaleIntensityRanged,
    CropForegroundd, RandFlipd, RandShiftIntensityd,
    EnsureTyped, Resized
)

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),  
    ScaleIntensityRanged(keys=["image"], a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
    RandShiftIntensityd(keys=["image"], offsets=0.10, prob=0.5),
    Resized(keys=["image", "label"], spatial_size=(128, 128, 64)),
    EnsureTyped(keys=["image", "label"])
])


<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-08-11 18:38:19.539079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754937499.720285      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754937499.771504      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Build MONAI Dataset & DataLoader

In [ ]:
from monai.data import Dataset, DataLoader

train_ds = Dataset(data=data_dicts[:10], transform=train_transforms)  # use fewer samples to test
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)


Define the SwinUNETR model

In [ ]:
for batch in train_loader:
    print("Max label value:", batch["label"].max().item())
    break


In [ ]:
from monai.networks.nets import SwinUNETR
import torch

device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

model = SwinUNETR(
    spatial_dims=3,
    in_channels=4,     # 4 MRI modalities: T1, T1ce, T2, FLAIR
    out_channels=5,    # For example: NCR/NET, ED, ET (segmentation labels)
    feature_size=24,
    use_checkpoint=False
).to(device)

Define loss, optimizer, and run 1 training step

In [ ]:
from monai.losses import DiceLoss
from torch.optim import Adam

loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = Adam(model.parameters(), 1e-4)

num_epochs = 5  # example
train_losses = []  # ✅ initialize here
val_losses = []    # if you do validation

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_loader:
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = model(inputs)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        step += 1

    avg_loss = epoch_loss / step
    train_losses.append(avg_loss)  # ✅ won't cause NameError
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")


Validation transforms

In [ ]:
val_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    ScaleIntensityRanged(keys=["image"], a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    Resized(keys=["image", "label"], spatial_size=(128, 128, 64)),
    EnsureTyped(keys=["image", "label"])
])

Split dataset

In [ ]:
train_data = data_dicts[:200]  # adjust as needed
val_data = data_dicts[200:220]  # adjust for your test size

train_ds = Dataset(data=train_data, transform=train_transforms)
val_ds = Dataset(data=val_data, transform=val_transforms)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete
from monai.data.utils import decollate_batch

# Metrics
dice_metric = DiceMetric(include_background=True, reduction="mean")
post_pred = AsDiscrete(argmax=True, to_onehot=5)  # 5 classes
post_label = AsDiscrete(to_onehot=5)


In [ ]:
def validate(model, val_loader):
    model.eval()
    dice_metric.reset()
    
    with torch.no_grad():
        for batch in val_loader:
            val_images = batch["image"].to(device)
            val_labels = batch["label"].to(device)
            
            val_outputs = model(val_images)
            
            # Convert outputs & labels to one-hot
            val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
            val_labels = [post_label(i) for i in decollate_batch(val_labels)]
            
            # Compute Dice
            dice_metric(y_pred=val_outputs, y=val_labels)
    
    mean_dice = dice_metric.aggregate().item()
    dice_metric.reset()
    return mean_dice


In [ ]:
max_epochs = 5  # change as needed

for epoch in range(max_epochs):
    print(f"Epoch {epoch+1}/{max_epochs}")
    epoch_loss = 0
    step = 0
    
    # --- Training ---
    model.train()
    for batch in train_loader:
        step += 1
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = model(inputs)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / step
    train_losses.append(avg_loss)

    # --- Validation ---
    val_dice = validate(model, val_loader)
    val_dices.append(val_dice)

    print(f"Train Loss: {avg_loss:.4f} | Val Dice: {val_dice:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))

# Loss plot
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss", marker='o')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# Dice score plot
plt.subplot(1,2,2)
plt.plot(val_dices, label="Val Dice", color='orange', marker='o')
plt.title("Validation Dice Score")
plt.xlabel("Epoch")
plt.ylabel("Dice Score")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
